In [ ]:
import pandas as pd
from phoenix.client import Client
import os

# os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = os.getenv("PHOENIX_COLLECTOR_ENDPOINT") or getpass.getpass('Phoenix Collector Endpoint:')
# os.environ["PHOENIX_API_KEY"] = os.getenv("PHOENIX_API_KEY") or getpass.getpass('Phoenix API Key:')

phoenix_client = Client()

dataset = pd.read_json("questions.json")

phoenix_dataset = phoenix_client.datasets.create_dataset(
    name="HotpotQA Final Answer Training",
    dataset_description="HotpotQA Final Answer Training",
    dataframe=dataset,
    input_keys=["question", "summary_1", "summary_2", "question_id"],
    output_keys=["gold_answer"],
)

In [2]:
import eval
import importlib
importlib.reload(eval)
from eval import evaluator

In [3]:
from ast import FormattedValue
from openai import AsyncOpenAI
from phoenix.client.experiments import async_run_experiment
from eval import evaluator

client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
system_prompt = "Given the fields {question}, {summary_1}, {summary_2}, produce an answer."

def make_task(system_prompt):
    async def task(input):
        formatted_prompt = system_prompt.replace("{question}", input["question"]).replace("{summary_1}", input["summary_1"]).replace("{summary_2}", input["summary_2"])
        response = await client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "user", "content": formatted_prompt}
            ]
        )
        return response.choices[0].message.content
    return task

experiment = await async_run_experiment(
    task=make_task(system_prompt),
    dataset=phoenix_dataset,
    evaluators=[evaluator],
    concurrency=40
)

🧪 Experiment started.
📺 View dataset experiments: http://127.0.0.1:6006/datasets/RGF0YXNldDo4/experiments
🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDo4/compare?experimentId=RXhwZXJpbWVudDoxMQ==


running tasks |██████████| 150/150 (100.0%) | ⏳ 00:13<00:00 | 10.89it/s


✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |██████████| 150/150 (100.0%) | ⏳ 01:39<00:00 |  1.50it/s

Experiment completed: 150 task runs, 1 evaluator runs, 150 evaluations


In [ ]:
import requests

def process_experiment(experiment, dataset):
    experiment_id = experiment["experiment_id"]
    url = f"http://localhost:6006/v1/experiments/{experiment_id}/json"
    headers = {
        "Authorization": f"Bearer {os.environ['PHOENIX_API_KEY']}"
    }

    response = requests.get(url, headers=headers)
    results = response.json()

    for i, entry in enumerate(results):
        question_id = entry["input"]["question_id"]
        eval_output = entry["annotations"][0]["explanation"]
        dataset.loc[question_id, "evaluation"] = eval_output

    return dataset

dataset = process_experiment(experiment, dataset)
print(dataset.columns)
dataset.head()

Index(['question', 'question_id', 'gold_answer', 'query_1', 'passages_1',
       'summary_1', 'query_2', 'passages_2', 'summary_2', 'final_answer',
       'evaluation'],
      dtype='object')


,question,question_id,gold_answer,query_1,passages_1,summary_1,query_2,passages_2,summary_2,final_answer,evaluation
5ae5decc5542996de7b71a37,Who was the widow affected by the Same-sex mar...,5ae5decc5542996de7b71a37,Jim Obergefell,Who was the widow affected by the Sixth Circui...,[{'title': 'Same-sex marriage in the Sixth Cir...,"The Sixth Circuit Court of Appeals, which incl...",Who was the widow affected by the Sixth Circui...,[{'title': 'Same-sex marriage in the Sixth Cir...,"The Sixth Circuit Court of Appeals, covering K...",The available information does not specify or ...,\n f1_score: 0.0\n precision: 0.0\n r...
5a8a295a55429930ff3c0cc8,Rowland Barnes was murdered by a man on trial ...,5a8a295a55429930ff3c0cc8,rape,Who was the man on trial for the crime of murd...,"[{'title': 'Who Killed Pasolini?', 'content': ...","Rowland Barnes, a Superior Court Judge in Fult...",For what crime was Brian Nichols on trial when...,"[{'title': 'Rowland Barnes', 'content': 'Rowla...","Rowland Barnes, a Superior Court Judge in Fult...","Rowland Barnes was murdered by Brian Nichols, ...",\n f1_score: 0.1\n precision: 0.05263157...
5adc46a055429944faac24b2,"When was the actor who stared in ""Shallow Hal""...",5adc46a055429944faac24b2,"August 28, 1969",What is the birthdate of the actor who starred...,"[{'title': 'Jack Black', 'content': 'Thomas Ja...","The actor who starred in ""Shallow Hal,"" ""Schoo...","When was Jack Black, the actor who starred in ...","[{'title': 'Jack Black', 'content': 'Thomas Ja...","The actor who starred in ""Shallow Hal,"" ""Schoo...","The actor who starred in ""Shallow Hal,"" ""Schoo...",\n f1_score: 0.18181818181818182\n preci...
5a88948f5542993e715ac04e,Scott Eastwood appeared in a 2014 American war...,5a88948f5542993e715ac04e,David Ayer,Who wrote and directed the 2014 American war f...,"[{'title': 'Fury (2014 film)', 'content': 'Fur...",Scott Eastwood appeared in the 2014 American w...,Who wrote and directed the 2014 American war f...,"[{'title': 'Fury (2014 film)', 'content': 'Fur...",Scott Eastwood appeared in the 2014 American w...,Scott Eastwood appeared in the 2014 American w...,\n f1_score: 0.14814814814814814\n preci...
5ab56b97554299637185c587,"HIStory: Past, Present and Future, Book I was ...",5ab56b97554299637185c587,King of Pop,"Who is the singer known as the ""King of Pop"" w...","[{'title': 'HIStory: Past, Present and Future,...","The album ""HIStory: Past, Present and Future, ...","Who was the singer known as ""The King of Pop"" ...","[{'title': 'HIStory: Past, Present and Future,...","HIStory: Past, Present and Future, Book I is t...","The singer who released the album ""HIStory: Pa...",\n f1_score: 0.24000000000000002\n preci...


In [17]:
import optimizer_sdk.tiktoken_splitter
importlib.reload(optimizer_sdk.tiktoken_splitter)
import optimizer_sdk.prompt_learning_optimizer
importlib.reload(optimizer_sdk.prompt_learning_optimizer)
from optimizer_sdk.prompt_learning_optimizer import PromptLearningOptimizer


In [18]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parents[3]
sys.path.insert(0, str(project_root))

from optimizer_sdk.prompt_learning_optimizer import PromptLearningOptimizer

optimizer = PromptLearningOptimizer(
    prompt=system_prompt,
    model_choice="gpt-4.1-mini",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

optimized_prompt = optimizer.optimize(
    dataset=dataset,
    output_column="final_answer",
    feedback_columns=["gold_answer", "evaluation"],
    context_size_k=100000
)

print(optimized_prompt)

I AM HERE
['question', 'question_id', 'gold_answer', 'query_1', 'passages_1', 'summary_1', 'query_2', 'passages_2', 'summary_2', 'final_answer', 'evaluation']

🔧 Creating batches with 100,000 token limit
📊 Processing 150 examples in 4 batches
   ✅ Batch 1/4: Optimized
   ✅ Batch 2/4: Optimized
   ✅ Batch 3/4: Optimized
   ✅ Batch 4/4: Optimized
Given the fields {question}, {summary_1}, {summary_2}, produce a concise and precise answer that directly and minimally responds to the question.  
- Provide the shortest possible answer that fully addresses the question.  
- If the question expects a specific name, date, number, phrase, or list, output only that without any additional explanation, context, or full sentences.  
- For yes/no questions, answer with "yes" or "no" only, without elaboration.  
- Avoid adding extra context, background information, or verbose sentences.  
- If the information is not available or cannot be determined from the summaries, respond with "No information avai